In [1]:
#Importing all the necessary libraries
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Activation, Input, concatenate
from keras.layers.pooling import MaxPool2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras import backend as K
K.set_image_data_format('channels_first') #Because our pretrained model uses channel first convention
from fr_utils import *
from inception_blocks_v2 import *
from PIL import Image

Using TensorFlow backend.


In [2]:
FRmodel = faceRecoModel(input_shape = (3, 96, 96))
print('Total parameters :',FRmodel.count_params())

Total parameters : 3743280


In [3]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    '''y_pred --> list contains three objects
    anchor -- shape : (None,128)
    positive -- shape : (None, 128)
    negative -- shape : (None , 128)
    '''
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    #Encoding distance between anchor and positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
    
    #Encoding distance between anchor and negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
    
    #Subtract the two previous distances and add alpha.
    difference = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    
    #Finding the maximum between difference and 0. And summing over the entire training examples
    loss = tf.reduce_sum(tf.maximum(difference, 0.0))
    
    return loss
  

In [4]:
#To check, if function performs well
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 528.1432


In [5]:
#Loading the trained model
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [6]:
#Reshaping images to (3,96,96)
path = 'images'
for img in os.listdir(path):
    img_array = cv2.imread(os.path.join(path,img))
    new_array = cv2.resize(img_array,(96,96))
    cv2.imwrite('modified_images/new_img_{}.jpg'.format(np.random.randint(1,1000)),new_array) #saving it to a new folder

Images can be converted to the required size using this way. Incase if you find the the converted images is of bad quality(in my case), you can reshape the image size manually.

In [7]:
#Creating a database to  store the names and its encodings
database = {}
database['Usain Bolt'] = img_to_encoding("images/bolt_96x96.jpg",FRmodel)
database['Cristiano Ronaldo'] = img_to_encoding("images/cristiano_1_96x96.jpg",FRmodel)
database['Lewis Hamilton'] = img_to_encoding("images/hamilton_96x96.jpg",FRmodel)
database['Virat Kohli'] = img_to_encoding("images/kohli_1_96x96.jpg",FRmodel)
database['Lionel Messi'] = img_to_encoding("images/messi_2_96x96.jpg",FRmodel)
database['Elon Musk'] = img_to_encoding("images/elon_96x96.png",FRmodel)
database['Jeff Bezos'] = img_to_encoding("images/jeff_96x96.jpg",FRmodel)
database['Robert Downey Jr'] = img_to_encoding("images/rdj_3_96x96.jpg",FRmodel)
database['Tom Cruise'] = img_to_encoding("images/tom_1_96x96.jpg",FRmodel)
database['Narendra Modi'] = img_to_encoding("images/modi_1_96x96.jpg",FRmodel)


# Face Verification

In [8]:
def verify(image_path, identity, database, model):
    #Compute the encoding of the image
    encoding = img_to_encoding(image_path, model)
    
    #Compute distance with identity's image
    dist = np.linalg.norm(encoding - database[identity])
    
    if dist < 0.65:
        print("Hey! " +str(identity)+", welcome to the party")
        door_open = True
    else:
        print("It's not "+str(identity)+" please go away")
        door_open = False
        
    return dist, door_open
  

In [9]:
verify('test_images/ronaldo_96x96.jpg', 'Cristiano Ronaldo', database, FRmodel)

Hey! Cristiano Ronaldo, welcome to the party


(0.6072807, True)

In [10]:
verify('test_images/trump_96x96.jpg', 'Virat Kohli', database, FRmodel)

It's not Virat Kohli please go away


(0.68979377, False)

# Face Recognition

In [11]:
def recognise(image_path, database, model):
    encoding = img_to_encoding(image_path, model)
    min_dist = 100
    
    for (name, db_enc) in database.items():
        dist = np.linalg.norm(encoding - db_enc)
        
        if dist < min_dist:
            min_dist = dist
            identity = name
            
    if min_dist > 0.65:
        print('Not in the database')
    else:
        print("It's "+str(identity)+", the distance is "+str(min_dist))
            
    return min_dist, identity

In [14]:
recognise('test_images/trump_96x96.jpg', database, FRmodel)

Not in the database


(0.68979377, 'Virat Kohli')

In [16]:
recognise('test_images/bolt_4_96x96.jpg', database, FRmodel)

It's Usain Bolt, the distance is 0.4575208


(0.4575208, 'Usain Bolt')